<a href="https://colab.research.google.com/github/Danazzz/dataset-beras-scm/blob/main/SCM_Beras1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [383]:
# Import libraries
import pandas as pd
import zipfile, os
import numpy as np
import csv
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime
from statsmodels.tsa.stattools import adfuller
plt.rcParams['figure.figsize'] = 15, 6            #agar semua plot ukurannya sama
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from sklearn.metrics import mean_squared_error
from math import sqrt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.vector_ar.var_model import VAR

import xgboost

In [384]:
#Load data BI
converters = {
    'Beras': lambda x: str(x.replace(',', '')),
    'Beras Kualitas Bawah I' : lambda x: str(x.replace(',', '')),
    'Beras Kualitas Medium I' : lambda x: str(x.replace(',', '')),
    'Beras Kualitas Super I' : lambda x: str(x.replace(',', '')),
}

df1 = pd.read_csv('./csv/Tabel Harga Berdasarkan Daerah Jabar.csv', converters=converters)
df1

,Komoditas,Beras,Beras Kualitas Bawah I,Beras Kualitas Bawah II,Beras Kualitas Medium I,Beras Kualitas Medium II,Beras Kualitas Super I,Beras Kualitas Super II,Beras Kualitas Bawah,Beras Kualitas Medium,Beras Kualitas Super
0,01/ 01/ 2020,-,-,-,-,-,-,-,0,0,0
1,02/ 01/ 2020,11150,10100,9500,11350,10850,12900,12100,9800,11100,12500
2,03/ 01/ 2020,11150,10100,9500,11350,10850,12900,12100,9800,11100,12500
3,06/ 01/ 2020,11150,10150,9550,11350,10850,12900,12100,9850,11100,12500
4,07/ 01/ 2020,11200,10200,9650,11400,10900,12950,12150,9925,11150,12550
...,...,...,...,...,...,...,...,...,...,...,...
1093,11/ 03/ 2024,15800,14950,14500,16050,15550,17250,16550,14725,15800,16900
1094,12/ 03/ 2024,15800,14950,14500,16050,15550,17250,16550,14725,15800,16900
1095,13/ 03/ 2024,15750,14900,14400,16000,15450,17200,16550,14650,15725,16875
1096,14/ 03/ 2024,15750,14900,14400,16000,15450,17200,16550,14650,15725,16875


In [385]:
print('Data Types:')
print(df1.dtypes)

Data Types:
Komoditas                   object
Beras                       object
Beras Kualitas Bawah I      object
Beras Kualitas Bawah II     object
Beras Kualitas Medium I     object
Beras Kualitas Medium II    object
Beras Kualitas Super I      object
Beras Kualitas Super II     object
Beras Kualitas Bawah        object
Beras Kualitas Medium       object
Beras Kualitas Super        object
dtype: object


In [386]:
#Mengubah tanggal menjadi index
con=df1['Komoditas']
df1['Komoditas'] = pd.to_datetime(df1['Komoditas'], format='%d/ %m/ %Y')
df1.set_index('Komoditas', inplace=True)
#check datatype of index
df1.index

DatetimeIndex(['2020-01-01', '2020-01-02', '2020-01-03', '2020-01-06',
               '2020-01-07', '2020-01-08', '2020-01-09', '2020-01-10',
               '2020-01-13', '2020-01-14',
               ...
               '2024-03-04', '2024-03-05', '2024-03-06', '2024-03-07',
               '2024-03-08', '2024-03-11', '2024-03-12', '2024-03-13',
               '2024-03-14', '2024-03-15'],
              dtype='datetime64[ns]', name='Komoditas', length=1098, freq=None)

In [387]:
df1 = df1.iloc[:, [0, 7, 8, 9]]
df1

,Beras,Beras Kualitas Bawah,Beras Kualitas Medium,Beras Kualitas Super
Komoditas,,,,
2020-01-01,-,0,0,0
2020-01-02,11150,9800,11100,12500
2020-01-03,11150,9800,11100,12500
2020-01-06,11150,9850,11100,12500
2020-01-07,11200,9925,11150,12550
...,...,...,...,...
2024-03-11,15800,14725,15800,16900
2024-03-12,15800,14725,15800,16900
2024-03-13,15750,14650,15725,16875


In [388]:
df1.info()
df1.shape

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1098 entries, 2020-01-01 to 2024-03-15
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Beras                  1098 non-null   object
 1   Beras Kualitas Bawah   1098 non-null   object
 2   Beras Kualitas Medium  1098 non-null   object
 3   Beras Kualitas Super   1098 non-null   object
dtypes: object(4)
memory usage: 42.9+ KB


(1098, 4)

In [389]:
converters = {
    'Luas Panen Padi': lambda x: str(x.replace('.', '')),
    'GKP Tingkat Petani': lambda x: str(x.replace('.', '')),
    'GKP Tingkat Penggilingan' : lambda x: str(x.replace('.', '')),
    'GKG Tingkat Penggilingan' : lambda x: str(x.replace('.', '')),
    'Beras Medium Penggilingan' : lambda x: str(x.replace('.', '')),
    'Beras Premium Penggilingan' : lambda x: str(x.replace('.', '')),
}

#Load data
df2 = pd.read_csv('./csv/export-produsen-2.csv', converters=converters)
# print(df2.head())
df2

,Komoditas,Luas Panen Padi,GKP Tingkat Petani,GKP Tingkat Penggilingan,GKG Tingkat Penggilingan,Beras Medium Penggilingan,Beras Premium Penggilingan,Ratarata Beras Penggilingan,Jagung Pipilan Kering,Kedelai Biji Kering (Lokal),Bawang Merah,Cabai Merah Keriting,Cabai Rawit Merah,Sapi (Hidup),Ayam Ras Pedaging (Hidup),Telur Ayam Ras,Stok GKG Tingkat Penggilingan,Stok Beras Tingkat Penggilingan
0,01/01/2020,20,3700,4000,4500,8200,9200,8700,-,-,-,-,-,-,-,-,600,150
1,02/01/2020,-,-,-,-,-,-,#DIV/0!,-,-,-,-,-,-,-,-,-,-
2,03/01/2020,-,-,-,-,-,-,#DIV/0!,-,-,-,-,-,-,-,-,-,-
3,04/01/2020,-,-,-,-,-,-,#DIV/0!,-,-,-,-,-,-,-,-,-,-
4,05/01/2020,-,-,-,-,-,-,#DIV/0!,-,-,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1531,11/03/2024,10,6760,7140,8330,14180,15640,14910,5540,10610,15990,41880,45050,54440,24480,27540,120,80
1532,12/03/2024,10,6530,6930,8260,14140,15480,14810,5250,10610,16330,42830,43390,54470,24480,28260,120,80
1533,13/03/2024,10,6480,6920,8240,14040,15460,14750,5280,10310,15990,43770,42270,54560,24000,28200,100,60
1534,14/03/2024,10,6510,7060,8240,14050,15510,14780,5280,10310,15840,45070,41840,54550,24010,28130,150,70


PRODUSEN
Panel Harga Pangan pada tingkat produsen yang diamati meliputi:
- Luas Lahan/Sawah Padi, persentase (%) luas panen Padi di kecamatan sentra produksi.
- GKP Tingkat Petani, harga di Petani (Rp/Kg)
- GKP Tingkat Penggilingan, harga di Penggilingan (Rp/Kg)
- GKG Tingkat Penggilingan, harga di Penggilingan (Rp/Kg)
- Beras Medium Tingkat Penggilingan, harga di Penggilingan (Rp/Kg)
- Beras Premium Tingkat Penggilingan, harga di Penggilingan (Rp/Kg)
- Stok GKG Tingkat Penggilingan, stok dalam satuan Kwintal
- Stok Beras Tingkat Penggilingan, stok dalam satuan Kwintal

In [390]:
print('Data Types:')
print(df2.dtypes)

Data Types:
Komoditas                          object
Luas Panen Padi                    object
GKP Tingkat Petani                 object
GKP Tingkat Penggilingan           object
GKG Tingkat Penggilingan           object
Beras Medium Penggilingan          object
Beras Premium Penggilingan         object
Ratarata Beras Penggilingan        object
Jagung Pipilan Kering              object
Kedelai Biji Kering (Lokal)        object
Bawang Merah                       object
Cabai Merah Keriting               object
Cabai Rawit Merah                  object
Sapi (Hidup)                       object
Ayam Ras Pedaging (Hidup)          object
Telur Ayam Ras                     object
Stok GKG Tingkat Penggilingan      object
Stok Beras Tingkat Penggilingan    object
dtype: object


In [391]:
#Mengubah tanggal menjadi index
con=df2['Komoditas']
df2['Komoditas'] = pd.to_datetime(df2['Komoditas'], format='%d/%m/%Y')
df2.set_index('Komoditas', inplace=True)
#check datatype of index
df2.index

DatetimeIndex(['2020-01-01', '2020-01-02', '2020-01-03', '2020-01-04',
               '2020-01-05', '2020-01-06', '2020-01-07', '2020-01-08',
               '2020-01-09', '2020-01-10',
               ...
               '2024-03-06', '2024-03-07', '2024-03-08', '2024-03-09',
               '2024-03-10', '2024-03-11', '2024-03-12', '2024-03-13',
               '2024-03-14', '2024-03-15'],
              dtype='datetime64[ns]', name='Komoditas', length=1536, freq=None)

In [392]:
print(df2.columns)

Index(['Luas Panen Padi', 'GKP Tingkat Petani', 'GKP Tingkat Penggilingan',
       'GKG Tingkat Penggilingan', 'Beras Medium Penggilingan',
       'Beras Premium Penggilingan', 'Ratarata Beras Penggilingan',
       'Jagung Pipilan Kering', 'Kedelai Biji Kering (Lokal)', 'Bawang Merah',
       'Cabai Merah Keriting', 'Cabai Rawit Merah', 'Sapi (Hidup)',
       'Ayam Ras Pedaging (Hidup)', 'Telur Ayam Ras',
       'Stok GKG Tingkat Penggilingan', 'Stok Beras Tingkat Penggilingan'],
      dtype='object')


In [393]:
df2 = df2.iloc[:, [0, 1, 2, 3, 6, 15, 16]]
df2

,Luas Panen Padi,GKP Tingkat Petani,GKP Tingkat Penggilingan,GKG Tingkat Penggilingan,Ratarata Beras Penggilingan,Stok GKG Tingkat Penggilingan,Stok Beras Tingkat Penggilingan
Komoditas,,,,,,,
2020-01-01,20,3700,4000,4500,8700,600,150
2020-01-02,-,-,-,-,#DIV/0!,-,-
2020-01-03,-,-,-,-,#DIV/0!,-,-
2020-01-04,-,-,-,-,#DIV/0!,-,-
2020-01-05,-,-,-,-,#DIV/0!,-,-
...,...,...,...,...,...,...,...
2024-03-11,10,6760,7140,8330,14910,120,80
2024-03-12,10,6530,6930,8260,14810,120,80
2024-03-13,10,6480,6920,8240,14750,100,60


In [394]:
# ini datanya ada yang gajelas 1
# luas panen 20710?
df2 = df2.drop(df2[df2['Luas Panen Padi'] == '20710'].index)

In [395]:
df2.info()
df2.shape

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1535 entries, 2020-01-01 to 2024-03-15
Data columns (total 7 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Luas Panen Padi                  1535 non-null   object
 1   GKP Tingkat Petani               1535 non-null   object
 2   GKP Tingkat Penggilingan         1535 non-null   object
 3   GKG Tingkat Penggilingan         1535 non-null   object
 4   Ratarata Beras Penggilingan      1535 non-null   object
 5   Stok GKG Tingkat Penggilingan    1535 non-null   object
 6   Stok Beras Tingkat Penggilingan  1535 non-null   object
dtypes: object(7)
memory usage: 95.9+ KB


(1535, 7)

In [396]:
df3 = pd.merge(df2, df1, on='Komoditas', how='left')
df3

,Luas Panen Padi,GKP Tingkat Petani,GKP Tingkat Penggilingan,GKG Tingkat Penggilingan,Ratarata Beras Penggilingan,Stok GKG Tingkat Penggilingan,Stok Beras Tingkat Penggilingan,Beras,Beras Kualitas Bawah,Beras Kualitas Medium,Beras Kualitas Super
Komoditas,,,,,,,,,,,
2020-01-01,20,3700,4000,4500,8700,600,150,-,0,0,0
2020-01-02,-,-,-,-,#DIV/0!,-,-,11150,9800,11100,12500
2020-01-03,-,-,-,-,#DIV/0!,-,-,11150,9800,11100,12500
2020-01-04,-,-,-,-,#DIV/0!,-,-,NaN,NaN,NaN,NaN
2020-01-05,-,-,-,-,#DIV/0!,-,-,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2024-03-11,10,6760,7140,8330,14910,120,80,15800,14725,15800,16900
2024-03-12,10,6530,6930,8260,14810,120,80,15800,14725,15800,16900
2024-03-13,10,6480,6920,8240,14750,100,60,15750,14650,15725,16875


In [397]:
df3.info()
df3.nunique()

# Menampilkan nilai unik dalam setiap kolom
for col in df3.columns:
    unique_values = df3[col].unique()
    print(f"Nilai unik dalam kolom {col}: {unique_values}")

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1535 entries, 2020-01-01 to 2024-03-15
Data columns (total 11 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Luas Panen Padi                  1535 non-null   object
 1   GKP Tingkat Petani               1535 non-null   object
 2   GKP Tingkat Penggilingan         1535 non-null   object
 3   GKG Tingkat Penggilingan         1535 non-null   object
 4   Ratarata Beras Penggilingan      1535 non-null   object
 5   Stok GKG Tingkat Penggilingan    1535 non-null   object
 6   Stok Beras Tingkat Penggilingan  1535 non-null   object
 7   Beras                            1097 non-null   object
 8   Beras Kualitas Bawah             1097 non-null   object
 9   Beras Kualitas Medium            1097 non-null   object
 10  Beras Kualitas Super             1097 non-null   object
dtypes: object(11)
memory usage: 143.9+ KB
Nilai unik dalam kolom Luas Panen Padi:

In [398]:
df4 = pd.read_csv('./csv/distributor_pembelian.csv')
df4

,Komoditas,UMP,BBM Solar,quantity
0,01/01/2020,"1,810,351","9,500","113,151"
1,02/01/2020,"1,810,351","9,500","113,151"
2,03/01/2020,"1,810,351","9,500","113,151"
3,04/01/2020,"1,810,351","9,500","113,151"
4,05/01/2020,"1,810,351","9,500","113,151"
...,...,...,...,...
1531,11/03/2024,"2,057,495","6,800","113,151"
1532,12/03/2024,"2,057,495","6,800","113,151"
1533,13/03/2024,"2,057,495","6,800","113,151"
1534,14/03/2024,"2,057,495","6,800","113,151"


In [399]:
#delete all commas in the data
df4['UMP'] = df4['UMP'].str.replace(',', '')
df4['BBM Solar'] = df4['BBM Solar'].str.replace(',', '')
df4['quantity'] = df4['quantity'].str.replace(',', '')
df4

,Komoditas,UMP,BBM Solar,quantity
0,01/01/2020,1810351,9500,113151
1,02/01/2020,1810351,9500,113151
2,03/01/2020,1810351,9500,113151
3,04/01/2020,1810351,9500,113151
4,05/01/2020,1810351,9500,113151
...,...,...,...,...
1531,11/03/2024,2057495,6800,113151
1532,12/03/2024,2057495,6800,113151
1533,13/03/2024,2057495,6800,113151
1534,14/03/2024,2057495,6800,113151


In [400]:
#Mengubah tanggal menjadi index
con=df4['Komoditas']
df4['Komoditas'] = pd.to_datetime(df4['Komoditas'], format='%d/%m/%Y')
df4.set_index('Komoditas', inplace=True)
#check datatype of index
df4.index

DatetimeIndex(['2020-01-01', '2020-01-02', '2020-01-03', '2020-01-04',
               '2020-01-05', '2020-01-06', '2020-01-07', '2020-01-08',
               '2020-01-09', '2020-01-10',
               ...
               '2024-03-06', '2024-03-07', '2024-03-08', '2024-03-09',
               '2024-03-10', '2024-03-11', '2024-03-12', '2024-03-13',
               '2024-03-14', '2024-03-15'],
              dtype='datetime64[ns]', name='Komoditas', length=1536, freq=None)

In [401]:
df4 = df4.astype('Int64')
df4

,UMP,BBM Solar,quantity
Komoditas,,,
2020-01-01,1810351,9500,113151
2020-01-02,1810351,9500,113151
2020-01-03,1810351,9500,113151
2020-01-04,1810351,9500,113151
2020-01-05,1810351,9500,113151
...,...,...,...
2024-03-11,2057495,6800,113151
2024-03-12,2057495,6800,113151
2024-03-13,2057495,6800,113151


In [402]:
df5 = pd.merge(df3, df4, on='Komoditas', how='left')
df5

,Luas Panen Padi,GKP Tingkat Petani,GKP Tingkat Penggilingan,GKG Tingkat Penggilingan,Ratarata Beras Penggilingan,Stok GKG Tingkat Penggilingan,Stok Beras Tingkat Penggilingan,Beras,Beras Kualitas Bawah,Beras Kualitas Medium,Beras Kualitas Super,UMP,BBM Solar,quantity
Komoditas,,,,,,,,,,,,,,
2020-01-01,20,3700,4000,4500,8700,600,150,-,0,0,0,1810351,9500,113151
2020-01-02,-,-,-,-,#DIV/0!,-,-,11150,9800,11100,12500,1810351,9500,113151
2020-01-03,-,-,-,-,#DIV/0!,-,-,11150,9800,11100,12500,1810351,9500,113151
2020-01-04,-,-,-,-,#DIV/0!,-,-,NaN,NaN,NaN,NaN,1810351,9500,113151
2020-01-05,-,-,-,-,#DIV/0!,-,-,NaN,NaN,NaN,NaN,1810351,9500,113151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-11,10,6760,7140,8330,14910,120,80,15800,14725,15800,16900,2057495,6800,113151
2024-03-12,10,6530,6930,8260,14810,120,80,15800,14725,15800,16900,2057495,6800,113151
2024-03-13,10,6480,6920,8240,14750,100,60,15750,14650,15725,16875,2057495,6800,113151


In [403]:
print(df5.dtypes)

Luas Panen Padi                    object
GKP Tingkat Petani                 object
GKP Tingkat Penggilingan           object
GKG Tingkat Penggilingan           object
Ratarata Beras Penggilingan        object
Stok GKG Tingkat Penggilingan      object
Stok Beras Tingkat Penggilingan    object
Beras                              object
Beras Kualitas Bawah               object
Beras Kualitas Medium              object
Beras Kualitas Super               object
UMP                                 Int64
BBM Solar                           Int64
quantity                            Int64
dtype: object


In [404]:
# Mengubah - menjadi 0
df5.replace('-', np.nan, inplace=True)
df5.replace('#DIV/0!', np.nan, inplace=True)
df5.dropna(inplace=True)
df5 = df5.astype('Int64')
df5


,Luas Panen Padi,GKP Tingkat Petani,GKP Tingkat Penggilingan,GKG Tingkat Penggilingan,Ratarata Beras Penggilingan,Stok GKG Tingkat Penggilingan,Stok Beras Tingkat Penggilingan,Beras,Beras Kualitas Bawah,Beras Kualitas Medium,Beras Kualitas Super,UMP,BBM Solar,quantity
Komoditas,,,,,,,,,,,,,,
2021-03-15,10,4000,4150,5050,9465,20,10,11200,9800,11150,12650,1810351,9600,113151
2021-03-16,10,3850,4100,4740,9360,70,20,11200,9800,11150,12650,1810351,9600,113151
2021-03-17,10,4160,4230,4950,9400,30,10,11200,9800,11150,12650,1810351,9600,113151
2021-03-18,10,4120,4190,4730,9340,40,10,11200,9775,11175,12650,1810351,9600,113151
2021-03-19,10,4190,4290,4990,9410,40,10,11200,9800,11150,12650,1810351,9600,113151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-11,10,6760,7140,8330,14910,120,80,15800,14725,15800,16900,2057495,6800,113151
2024-03-12,10,6530,6930,8260,14810,120,80,15800,14725,15800,16900,2057495,6800,113151
2024-03-13,10,6480,6920,8240,14750,100,60,15750,14650,15725,16875,2057495,6800,113151
